In [43]:
import gspread
import pandas as pd
import json
import os
from gspread_dataframe import get_as_dataframe, set_with_dataframe


def get_gspread_client():
    """Get gspread client using either file or environment variable"""
    gdrive_creds_json = os.getenv("GDRIVE_CREDS_JSON")
    
    if gdrive_creds_json:
        print("Using environment variable for credentials")
        try:
            creds_dict = json.loads(gdrive_creds_json)
            return gspread.service_account_from_dict(creds_dict)
        except json.JSONDecodeError as e:
            print(f"Error parsing GDRIVE_CREDS_JSON: {e}")
            raise
    else:
        print("Using file for credentials")
        return gspread.service_account(filename='gdrive-creds.json')


def get_or_create_worksheet(sh, title, rows=1000, cols=26):
    """Get worksheet by name, or create if it does not exist."""
    try:
        return sh.worksheet(title)
    except gspread.exceptions.WorksheetNotFound:
        print(f"Creating new worksheet: {title}")
        return sh.add_worksheet(title=title, rows=rows, cols=cols)

In [44]:
from gspread_dataframe import get_as_dataframe

# Connect
gc = get_gspread_client()
sh = gc.open("myportfolio")

# Get trades and positions
ws_trades = sh.worksheet("flex-trades")
ws_positions = sh.worksheet("flex-positions")

trades = get_as_dataframe(ws_trades, evaluate_formulas=True, na_filter=False)
positions = get_as_dataframe(ws_positions, evaluate_formulas=True, na_filter=False)

# Clean empty rows
trades = trades.dropna(how='all')
positions = positions.dropna(how='all')

positions_summary = positions[positions['levelOfDetail'] == 'SUMMARY'].copy()
positions_lot = positions[positions['levelOfDetail'] == 'LOT'].copy()




Using file for credentials


In [65]:
import pandas as pd
import numpy as np

# Step 1: Load data
positions = get_as_dataframe(ws_positions, evaluate_formulas=True, na_filter=False)

positions['openDateTime'] = positions['openDateTime'].astype(str).replace('nan', '')

# Step 2: Parse openDateTime from IBKR format
def parse_ibkr_datetime(val):
    try:
        if pd.isnull(val):
            return pd.NaT
        val = str(val)
        if ';' in val and len(val) >= 15:
            return pd.to_datetime(val, format='%Y%m%d;%H%M%S')
        elif len(val) == 8 and val.isdigit():
            return pd.to_datetime(val, format='%Y%m%d')
        else:
            return pd.NaT
    except Exception:
        return pd.NaT

positions['openDateTime'] = positions['openDateTime'].apply(parse_ibkr_datetime)

# Step 3: Clean up levelOfDetail and ensure LOT rows come before SUMMARY
positions['levelOfDetail'] = positions['levelOfDetail'].str.strip().str.upper()
positions['levelOfDetail_order'] = positions['levelOfDetail'].map({'LOT': 0, 'SUMMARY': 1, '': 2})

# Step 4: Ensure all relevant columns are numeric
num_cols = [
    'positionValue', 'markPrice', 'costBasisPrice', 'position',
    'multiplier', 'fxRateToBase', 'fifoPnlUnrealized'
]
for col in num_cols:
    positions[col] = pd.to_numeric(positions[col], errors='coerce')

# Step 5: Sort so LOTs are before SUMMARY by symbol
positions = positions.sort_values(by=['symbol', 'levelOfDetail_order'])

# Step 6: Backfill openDateTime from LOT to SUMMARY by symbol
positions['openDateTime'] = positions.groupby('symbol')['openDateTime'].ffill()

# Step 7: Filter only SUMMARY rows
positions_summary = positions[positions['levelOfDetail'] == 'SUMMARY'].copy()

# Step 8: Calculate portfolio metrics
positions_summary['WeightInPortfolio'] = (positions_summary['positionValue'] / positions_summary['positionValue'].sum()) * 100
positions_summary['UnrealizedReturnPct'] = ((positions_summary['markPrice'] - positions_summary['costBasisPrice']) / positions_summary['costBasisPrice']) * 100
positions_summary['UnrealizedPnL'] = (positions_summary['markPrice'] - positions_summary['costBasisPrice']) * positions_summary['position'] * positions_summary['multiplier']

positions_summary['days_held'] = (pd.Timestamp.today() - positions_summary['openDateTime']).dt.days

positions_summary['AnnualizedReturn'] = np.where(
    positions_summary['days_held'] > 0,
    ((positions_summary['markPrice'] / positions_summary['costBasisPrice']) ** (365 / positions_summary['days_held']) - 1) * 100,
    np.nan
)
positions_summary['AnnualizedReturn'] = positions_summary['AnnualizedReturn'].replace([np.inf, -np.inf], np.nan)

# Step 9: Ensure metrics are numeric and rounded
calc_cols = ['WeightInPortfolio', 'UnrealizedReturnPct', 'AnnualizedReturn', 'UnrealizedPnL']
for col in calc_cols:
    positions_summary[col] = pd.to_numeric(positions_summary[col], errors='coerce').round(0)

# Optional: Clean up temp column
positions_summary = positions_summary.drop(columns=['levelOfDetail_order'], errors='ignore')

# Step 10: Display result
positions_summary


,accountId,acctAlias,model,currency,fxRateToBase,assetCategory,subCategory,symbol,description,conid,...,deliveryType,commodityType,fineness,weight,position_key,WeightInPortfolio,UnrealizedReturnPct,UnrealizedPnL,days_held,AnnualizedReturn
0,U19575352,,,USD,1.0,STK,COMMON,ACHR,ARCHER AVIATION INC-A,514640214,...,,,0,0,U19575352_514640214_20250626,8.0,5.0,23.0,1,1.291065e+09
2,U19575352,,,USD,1.0,STK,COMMON,AMZN,AMAZON.COM INC,3691937,...,,,0,0,U19575352_3691937_20250626,16.0,4.0,43.0,4,3.810000e+03
4,U19575352,,,USD,1.0,STK,COMMON,GOOG,ALPHABET INC-CL C,208813720,...,,,0,0,U19575352_208813720_20250626,26.0,5.0,85.0,4,9.698000e+03
7,U19575352,,,USD,1.0,STK,ETF,IB01,ISHARES US TREAS 0-1YR USD A,354802220,...,,,0,0,U19575352_354802220_20250626,19.0,-0.0,-1.0,4,-7.000000e+00
10,U19575352,,,USD,1.0,STK,COMMON,IBKR,INTERACTIVE BROKERS GRO-CL A,43645865,...,,,0,0,U19575352_43645865_20250626,2.0,3.0,3.0,22,7.100000e+01
12,U19575352,,,USD,1.0,STK,ETF,IBTA,ISHARES USD TRSRY 1-3Y USD A,272686955,...,,,0,0,U19575352_272686955_20250626,8.0,-0.0,-1.0,4,-1.200000e+01
15,U19575352,,,USD,1.0,STK,ETF,IGLN,ISHARES PHYSICAL GOLD ETC,86656182,...,,,0,0,U19575352_86656182_20250626,7.0,-2.0,-11.0,4,-8.600000e+01
18,U19575352,,,USD,1.0,STK,ETF,IWDA,ISHARES CORE MSCI WORLD,78999785,...,,,0,0,U19575352_78999785_20250626,6.0,2.0,7.0,4,3.980000e+02
21,U19575352,,,USD,1.0,STK,ETF,VUSD,VANG S&P500 USDD,107968733,...,,,0,0,U19575352_107968733_20250626,8.0,2.0,9.0,4,3.950000e+02


In [40]:
print(positions['levelOfDetail'].unique())


['' 'LOT' 'SUMMARY']


In [52]:
print(positions[positions['symbol'] == 'AMZN'][['symbol', 'levelOfDetail', 'openDateTime']])


  symbol levelOfDetail        openDateTime
3   AMZN           LOT 2025-06-23 15:45:46
2   AMZN       SUMMARY                 NaT


In [54]:
positions['openDateTime'] = positions.groupby('symbol')['openDateTime'].fill()
print(positions[positions['symbol'] == 'AMZN'][['symbol', 'levelOfDetail', 'openDateTime']])


  symbol levelOfDetail        openDateTime
3   AMZN           LOT 2025-06-23 15:45:46
2   AMZN       SUMMARY 2025-06-23 15:45:46
